**Author: Alexander Kollmann, 09/2022**

---

**Funktion**

FHIR - Abfragen und Suchen (nativ und mittels fhirclient)

*   einfache Suche über im Standard definierte Such-Parameter (? bzw. &)
*   Verkettete Suche über Referenzen
*   Inkludieren von Referenzierten Ressourcen in das Ergebnis
*   AddOn (z.B. Konvertierung des Suchergebnisses (JSON Bundle) in ein Pandas Dataframe)




---



**Referenzen**

https://learn.microsoft.com/en-us/azure/healthcare-apis/fhir/search-samples

https://cloud.google.com/healthcare-api/docs/how-tos/fhir-advanced-search

https://developer.commure.com/docs/apis/fhir/search-parameters/

https://colab.research.google.com/drive/1OSuqqACnCqw8h67E7DKDMyIGO1C2qbEq?usp=sharing

https://developer.commure.com/docs/apis/fhir/search-parameters/


---


In [1]:
import json
import requests
from collections import OrderedDict
from io import StringIO
from IPython.display import IFrame

In [2]:
# Define Base URL
url = "http://hapi.fhir.org/baseR4/" # Open HAPI FHIR Server
#url = "https://server.fire.ly/R4/" # Open Firely Server
headers = {"Content-Type": "application/fhir+json;charset=utf-8"}

In [3]:
# now add some search parameters

# each FHIR resource type has its own set of parameters; find them toward the bottom of the page for that resource type in the specification
# for example, for the Patient resource type, see https://www.hl7.org/fhir/patient.html#search
IFrame('https://www.hl7.org/fhir/patient.html#search', width=1200, height=500)

# Einfache Suche

In [4]:
# Abfrage nach Name UND Geburtsdatum
# Erwartetes Ergebnis: 1 Treffer

req = url + "Patient?" + "birthdate" + "=" + "2022-12-05" + "&" + "name" + "=" + "WienerNeustadtNJrLm"
#req = url + "Patient?" + "birthdate" + "=" + "2022-12-05"
print(req)
response = requests.request("GET", req, headers=headers)
result = response.json()
patient_id = result['entry'][0]['resource']['id']
print(response.text)

http://hapi.fhir.org/baseR4/Patient?birthdate=2022-12-05&name=WienerNeustadtNJrLm
{
  "resourceType": "Bundle",
  "id": "6a8f00ea-8f14-4690-a403-9b973c2da851",
  "meta": {
    "lastUpdated": "2022-12-13T14:25:51.556+00:00"
  },
  "type": "searchset",
  "total": 1,
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Patient?birthdate=2022-12-05&name=WienerNeustadtNJrLm"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Patient/7185197",
    "resource": {
      "resourceType": "Patient",
      "id": "7185197",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2022-12-05T15:33:01.605+00:00",
        "source": "#2eopjhS2NavITyoA"
      },
      "text": {
        "status": "generated",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><div class=\"hapiHeaderText\">Fachhochschule <b>WIENERNEUSTADTNJRLM </b></div><table class=\"hapiPropertyTable\"><tbody><tr><td>Date of birth</td><td><span>05 December 2022</span></td></tr><

In [5]:
#print(response.text)
patient_id

'7185197'

In [6]:
# Abfrage der Condition zu einem Patienten

# Teatpatient: MrXXX
#patient_id = str(7168516)
# Get Condition of this patient
req = url + "Condition?" + "patient=" + patient_id
response = requests.request("GET", req, headers=headers)
print("Anzahl: " + str(json.loads(response.text)['total']))
print(response.text)

Anzahl: 2
{
  "resourceType": "Bundle",
  "id": "32f76a44-f22c-4826-84b6-37d11b2fe593",
  "meta": {
    "lastUpdated": "2022-12-13T14:25:51.674+00:00"
  },
  "type": "searchset",
  "total": 2,
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Condition?patient=7185197"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Condition/7185206",
    "resource": {
      "resourceType": "Condition",
      "id": "7185206",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2022-12-05T15:39:04.904+00:00",
        "source": "#1HGvYIjOmEpnmBfq"
      },
      "code": {
        "coding": [ {
          "system": "http://hl7.org/fhir/sid/icd-10-us",
          "code": "E10.65",
          "display": "Type 1 Diabetes Mellitus with Hyperglycemia"
        } ]
      },
      "subject": {
        "reference": "Patient/7185197"
      },
      "encounter": {
        "reference": "Encounter/7185198"
      }
    },
    "search": {
      "mode": "match"


In [7]:
# Search Observations with the term ‘Cholesterol’’ in text description.

req = url + "Observation?_text=Cholesterol"
response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "Bundle",
  "id": "c8855597-fb80-42c1-87d2-c01529e9b4d3",
  "meta": {
    "lastUpdated": "2022-12-13T14:25:51.729+00:00"
  },
  "type": "searchset",
  "total": 5,
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Observation?_text=Cholesterol"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Observation/39402",
    "resource": {
      "resourceType": "Observation",
      "id": "39402",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2019-10-01T18:39:29.193+00:00",
        "source": "#9e8319c22dbc9c4d"
      },
      "text": {
        "status": "generated",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><p><b>Generated Narrative with Details</b></p><p><b>id</b>: ldlcholesterol</p><p><b>status</b>: final</p><p><b>code</b>: LDL Chol. (Calc) <span>(Details : {LOINC code '13457-7' = 'Cholesterol in LDL [Mass/volume] in Serum or Plasma by calculation', given as 'Cholesterol in LDL [Mass/volume] 

In [8]:
# alle Patienten mit Namen
req = url + "Patient?name=WienerNeustadt&_summary=count"
response = requests.request("GET", req, headers=headers)
print("Anzahl der Patienten: " + str(response.json().get('total')))

req = url + "Patient?name=WienerNeustadt"
response = requests.request("GET", req, headers=headers)
print(response.text)

Anzahl der Patienten: 9
{
  "resourceType": "Bundle",
  "id": "ed31c8d4-c101-4b45-9475-30c3fdca3049",
  "meta": {
    "lastUpdated": "2022-12-13T14:25:52.298+00:00"
  },
  "type": "searchset",
  "total": 9,
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Patient?name=WienerNeustadt"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Patient/7198903",
    "resource": {
      "resourceType": "Patient",
      "id": "7198903",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2022-12-13T14:21:01.360+00:00",
        "source": "#79Y22jJNcs9cOtNc"
      },
      "text": {
        "status": "generated",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><div class=\"hapiHeaderText\">Fachhochschule <b>WIENERNEUSTADTFJEDR </b></div><table class=\"hapiPropertyTable\"><tbody><tr><td>Date of birth</td><td><span>05 December 2022</span></td></tr></tbody></table></div>"
      },
      "name": [ {
        "use": "official",
       

In [9]:
# Anzahl der Patienten
req = url + "Patient?_summary=count"
response = requests.request("GET", req, headers=headers)
print("Anzahl der Patienten: " + str(response.json().get('total')))
print(response.text)

Anzahl der Patienten: 307718
{
  "resourceType": "Bundle",
  "id": "1ae33d6c-cfe2-497c-be2a-d9c111904e43",
  "meta": {
    "lastUpdated": "2022-12-13T14:25:52.825+00:00",
    "tag": [ {
      "system": "http://terminology.hl7.org/CodeSystem/v3-ObservationValue",
      "code": "SUBSETTED",
      "display": "Resource encoded in summary mode"
    } ]
  },
  "type": "searchset",
  "total": 307718
}


In [10]:
# Anzahl der Patienten
req = url + "Patient?_total=accurate"
response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "Bundle",
  "id": "b5edf11a-e744-4b13-8221-bfb070711758",
  "meta": {
    "lastUpdated": "2022-12-13T14:25:56.489+00:00"
  },
  "type": "searchset",
  "total": 307718,
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Patient?_total=accurate"
  }, {
    "relation": "next",
    "url": "https://hapi.fhir.org/baseR4?_getpages=b5edf11a-e744-4b13-8221-bfb070711758&_getpagesoffset=20&_count=20&_pretty=true&_bundletype=searchset"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Patient/257017",
    "resource": {
      "resourceType": "Patient",
      "id": "257017",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2019-12-12T11:45:41.379+00:00",
        "source": "#Hylk1jA0siRKUsx3"
      },
      "text": {
        "status": "generated",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\">\n      <table> \n        <tbody> \n          <tr> \n            <td> Name</td> \n            <td> Peter James \n  

In [11]:
# Rückgabe von Summary-Paramtern einer Ressource (idF. Patient)
req = url + "Patient?_summary=true"
# Rückgabe von ausgewählten Attributen einer Ressource (idF. Ressource Patient mit den Attributen id und gender)
#req = url + "Patient?_elements=id,gender"
response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "Bundle",
  "id": "8503dc85-590b-48da-8557-953a4176541e",
  "meta": {
    "lastUpdated": "2022-12-13T14:25:58.544+00:00",
    "tag": [ {
      "system": "http://terminology.hl7.org/CodeSystem/v3-ObservationValue",
      "code": "SUBSETTED",
      "display": "Resource encoded in summary mode"
    } ]
  },
  "type": "searchset",
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Patient?_summary=true"
  }, {
    "relation": "next",
    "url": "https://hapi.fhir.org/baseR4?_getpages=8503dc85-590b-48da-8557-953a4176541e&_getpagesoffset=20&_count=20&_pretty=true&_bundletype=searchset"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Patient/257017",
    "resource": {
      "resourceType": "Patient",
      "id": "257017",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2019-12-12T11:45:41.379+00:00",
        "source": "#Hylk1jA0siRKUsx3",
        "tag": [ {
          "system": "http://terminology.hl7.org/Code

In [12]:
# Anzahl der Patienten ohne GebDat
req = url + "Patient?birthdate:missing&_summary=count"

#Anzahl der Patienten mit Sterbekennzeichen
#req = url + "Patient?deceased:true&_summary=count"
response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "Bundle",
  "id": "bfef27f2-7a9f-4965-a28a-6363a66ab358",
  "meta": {
    "lastUpdated": "2022-12-13T14:25:58.656+00:00",
    "tag": [ {
      "system": "http://terminology.hl7.org/CodeSystem/v3-ObservationValue",
      "code": "SUBSETTED",
      "display": "Resource encoded in summary mode"
    } ]
  },
  "type": "searchset",
  "total": 202809
}


In [13]:
# Anzahl der Geschlechter und Abfragedauer
req = url + "Patient" + '?' + 'gender=male' + '&' + '_summary=count'
response = requests.request("GET", req, headers=headers)
print("male: " + str(response.json().get('total')) + " (" + str(response.elapsed.total_seconds()) + ' s)')
req = url + "Patient" + '?' + 'gender=female' + '&' + '_summary=count'
response = requests.request("GET", req, headers=headers)
print("female: " + str(response.json().get('total')) + " (" + str(response.elapsed.total_seconds()) + ' s)')
req = url + "Patient" + '?' + 'gender:missing=true' + '&' + '_summary=count'
response = requests.request("GET", req, headers=headers)
print("missing: " + str(response.json().get('total')) + " (" + str(response.elapsed.total_seconds()) + ' s)')

male: 147703 (9.197751 s)
female: 68297 (3.303723 s)
missing: None (20.111875 s)


In [14]:
# String Search
req = url + 'Patient' + '?' + 'family=Koll' + '&_elements=name'
#req = url + 'Patient' + '?' + 'family:exact=MrXXX' + '&_elements=name'
#req = url + 'Patient' + '?' + 'family:contains=Mr' + '&_elements=name'
response = requests.request("GET", req, headers=headers)
#print(response.text)

for entry in response.json().get('entry'):
    resource = entry.get('resource')
    print(resource.get('id'), end=': ')
    print(', '.join(map(lambda n: n.get('family'), resource.get('name'))))

2376392: Kölle
1709914: Kollege
7167994: Koller
7170339: Koller
7162763: Kolli
7162765: Kolli
7162767: Kolli
7162769: Kolli
7162771: Kolli
7162773: Kolli
7162777: Kolli
7162779: Kolli
7162781: Kolli
7162783: Kolli
7162785: Kolli
7162787: Kolli
7162789: Kolli
7162915: Kolli
7163145: Kolli
7163147: Kolli


In [15]:
IFrame('https://www.hl7.org/fhir/search.html#prefix', width=1200, height=500)

In [16]:
# Date Search
# date searches support lt(<), le(<=), gt(>), ge(>=), sa(starts after), and eb(ends before) "prefixes"

# Patienten mit Geburtsjahr
#req = url + 'Patient' + '?' + 'birthdate=1984'

# Patienten mit Geburtstag zwischen zwei Zeitpunkten
req = url + 'Patient?birthdate=gt2016-01-01&birthdate=lt2019-01-01'
response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "Bundle",
  "id": "2e832ada-e3a6-463c-90bf-215579f402d2",
  "meta": {
    "lastUpdated": "2022-12-13T14:26:38.055+00:00"
  },
  "type": "searchset",
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Patient?birthdate=gt2016-01-01&birthdate=lt2019-01-01"
  }, {
    "relation": "next",
    "url": "https://hapi.fhir.org/baseR4?_getpages=2e832ada-e3a6-463c-90bf-215579f402d2&_getpagesoffset=20&_count=20&_pretty=true&_bundletype=searchset"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Patient/1727761",
    "resource": {
      "resourceType": "Patient",
      "id": "1727761",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2021-01-05T13:00:21.487+00:00",
        "source": "#fJnbOVkf30U4fL52"
      },
      "text": {
        "status": "generated",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><div class=\"hapiHeaderText\">Test Name </div><table class=\"hapiPropertyTable\"><tbody><tr><td>Identi

In [17]:
IFrame('https://build.fhir.org/observation.html#search', width=1200, height=500)

In [18]:
# Range Search

code = "8867-4"

req = url + "/Observation?code=" + code + "&value-quantity=ge50&value-quantity=le200"
response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "Bundle",
  "id": "83a9cf3d-433e-4fbc-afd7-c9a5b9525b71",
  "meta": {
    "lastUpdated": "2022-12-13T14:26:39.846+00:00"
  },
  "type": "searchset",
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Observation?code=8867-4&value-quantity=ge50&value-quantity=le200"
  }, {
    "relation": "next",
    "url": "https://hapi.fhir.org/baseR4?_getpages=83a9cf3d-433e-4fbc-afd7-c9a5b9525b71&_getpagesoffset=20&_count=20&_pretty=true&_bundletype=searchset"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Observation/30195",
    "resource": {
      "resourceType": "Observation",
      "id": "30195",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2019-09-26T14:34:43.313+00:00",
        "source": "#d0126f9b8578248b"
      },
      "text": {
        "status": "generated",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\">Heart Rate, 78 bpm</div>"
      },
      "status": "final",
      "code": {
        "co

In [19]:
# Reference search
# Alle Patienten zurückgeben, die einen GP haben und den Namen/ Org des GP

req = url + 'Patient?general-practitioner:missing=false&_elements=id,name,generalPractitioner,managingOrganization'
response = requests.request("GET", req, headers=headers)
print(response.text)


{
  "resourceType": "Bundle",
  "id": "fad2b1a7-5c77-440d-b3a4-d05680db73ed",
  "meta": {
    "lastUpdated": "2022-12-13T14:26:41.940+00:00"
  },
  "type": "searchset",
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Patient?_elements=id%2Cname%2CgeneralPractitioner%2CmanagingOrganization&general-practitioner%3Amissing=false"
  }, {
    "relation": "next",
    "url": "https://hapi.fhir.org/baseR4?_getpages=fad2b1a7-5c77-440d-b3a4-d05680db73ed&_getpagesoffset=20&_count=20&_pretty=true&_bundletype=searchset&_elements=generalPractitioner,id,managingOrganization,name"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Patient/7197634",
    "resource": {
      "resourceType": "Patient",
      "id": "7197634",
      "meta": {
        "versionId": "6",
        "lastUpdated": "2022-12-13T10:22:29.468+00:00",
        "source": "#TQmmEai6yV349iEo",
        "tag": [ {
          "system": "http://terminology.hl7.org/CodeSystem/v3-ObservationValue",
  

In [20]:
#req = url + "Patient$export"

In [21]:
# Alle Conditions zu einem Patienten

req = url + 'Condition' + '?' + 'patient=' + url + '/Patient/' + patient_id + '&_elements=code'
response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "Bundle",
  "id": "5610f3f8-ff1b-4a23-b11f-087f2c6d83a6",
  "meta": {
    "lastUpdated": "2022-12-13T14:26:42.195+00:00"
  },
  "type": "searchset",
  "total": 0,
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Condition?_elements=code&patient=http%3A%2F%2Fhapi.fhir.org%2FbaseR4%2F%2FPatient%2F7185197"
  } ]
}


# Chained search

In [22]:
req = url + 'Encounter?subject:Patient.name=WienerNeustadt'
response = requests.request("GET", req, headers=headers)
print(str(response.text))

{
  "resourceType": "Bundle",
  "id": "dab154b6-bd75-465a-bf2f-52ad53c8291f",
  "meta": {
    "lastUpdated": "2022-12-13T14:26:46.493+00:00"
  },
  "type": "searchset",
  "total": 8,
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Encounter?subject%3APatient.name=WienerNeustadt"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Encounter/7185198",
    "resource": {
      "resourceType": "Encounter",
      "id": "7185198",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2022-12-05T15:33:02.413+00:00",
        "source": "#KgBioZMGvDDJVma6"
      },
      "text": {
        "status": "additional",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\">Encounter zum Patienten:[{'use': 'official', 'text': 'Fachhochschule WienerNeustadtNJrLm', 'family': 'WienerNeustadtNJrLm', 'given': ['Fachhochschule']}]</div>"
      },
      "status": "finished",
      "class": {
        "system": "http://terminology.hl7.org/CodeSystem/v

In [23]:
req = url + "Condition" + "?" + "subject:Patient.name=WienerNeustadt" + "&" + "code=http://hl7.org/fhir/sid/icd-10-us|E10.65" + "&" + "subject:Patient.gender=female&_summary=count"
response = requests.request("GET", req, headers=headers)
print(str(response.text))

{
  "resourceType": "Bundle",
  "id": "6ad9cd3d-fa60-46b0-ba4a-d68e7f2909b2",
  "meta": {
    "lastUpdated": "2022-12-13T14:26:48.819+00:00",
    "tag": [ {
      "system": "http://terminology.hl7.org/CodeSystem/v3-ObservationValue",
      "code": "SUBSETTED",
      "display": "Resource encoded in summary mode"
    } ]
  },
  "type": "searchset",
  "total": 6
}


In [24]:
req = url + 'DiagnosticReport?subject:Patient.name=Alex'
response = requests.request("GET", req, headers=headers)
print(str(response.text))

{
  "resourceType": "Bundle",
  "id": "fa5b7090-52ac-410c-97c4-0fb3c8896e76",
  "meta": {
    "lastUpdated": "2022-12-13T14:26:57.961+00:00"
  },
  "type": "searchset",
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/DiagnosticReport?subject%3APatient.name=Alex"
  }, {
    "relation": "next",
    "url": "https://hapi.fhir.org/baseR4?_getpages=fa5b7090-52ac-410c-97c4-0fb3c8896e76&_getpagesoffset=20&_count=20&_pretty=true&_bundletype=searchset"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/DiagnosticReport/2542218",
    "resource": {
      "resourceType": "DiagnosticReport",
      "id": "2542218",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2021-09-03T16:51:12.718+00:00",
        "source": "#8sr68niVvnDncWaA"
      },
      "text": {
        "status": "generated",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><div class=\"hapiHeaderText\"> HEMOGLOBIN A1C </div><table class=\"hapiPropertyTable\"><tbody

In [25]:
# Alle Diagnosen von Patienten mit Geburtsjahr 1930
req = url + 'Condition?patient.birthdate=1930'
response = requests.request("GET", req, headers=headers)
print(str(response.text))

{
  "resourceType": "Bundle",
  "id": "146a63d2-7ddd-4fc9-bbbc-5bbe1523977e",
  "meta": {
    "lastUpdated": "2022-12-13T14:26:58.854+00:00"
  },
  "type": "searchset",
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Condition?patient.birthdate=1930"
  }, {
    "relation": "next",
    "url": "https://hapi.fhir.org/baseR4?_getpages=146a63d2-7ddd-4fc9-bbbc-5bbe1523977e&_getpagesoffset=20&_count=20&_pretty=true&_bundletype=searchset"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Condition/6966594",
    "resource": {
      "resourceType": "Condition",
      "id": "6966594",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2022-08-22T21:18:20.961+00:00",
        "source": "#wIhXIGKf5Hsjs3df"
      },
      "code": {
        "coding": [ {
          "system": "http://snomed.info/sct",
          "code": "73211009",
          "display": "Diabetes mellitus (disorder)"
        } ],
        "text": "Facture of radius"
      },
  

# Reverse chained search

In [26]:
# Search for all Patients with Diabetes Mellitus

req = url + 'Patient' + '?' + '_has:Condition:patient:code=http://snomed.info/sct|44054006' + '&_summary=count'
response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "Bundle",
  "id": "11942c9a-cbed-4d69-8bcd-62315fc32f4d",
  "meta": {
    "lastUpdated": "2022-12-13T14:27:00.133+00:00",
    "tag": [ {
      "system": "http://terminology.hl7.org/CodeSystem/v3-ObservationValue",
      "code": "SUBSETTED",
      "display": "Resource encoded in summary mode"
    } ]
  },
  "type": "searchset",
  "total": 82
}


In [27]:
# Anzahl der Patienten mit einer Observation "Heart Rate"
# https://loinc.org/8867-4/

code = "8867-4"

req = url + "Patient?_has:Observation:patient:code=" + code + "&_summary=count"
response = requests.request("GET", req, headers=headers)
print("Anzahl der Patienten: " + str(response.json().get('total')))
print(response.text)

Anzahl der Patienten: 5845
{
  "resourceType": "Bundle",
  "id": "c2e834e8-d50c-4fdb-80de-3b1d179008c9",
  "meta": {
    "lastUpdated": "2022-12-13T14:27:00.488+00:00",
    "tag": [ {
      "system": "http://terminology.hl7.org/CodeSystem/v3-ObservationValue",
      "code": "SUBSETTED",
      "display": "Resource encoded in summary mode"
    } ]
  },
  "type": "searchset",
  "total": 5845
}


In [28]:
# Alle Patienten zurück geben mit einer gewissen Observation (idF. Heart Rate)
# Wenn es viele Treffer gibt, dann wird "Pageing" aktiviert.

req = url + "Patient?_has:Observation:patient:code=8867-4"

# die ersten 5 Ergebnisse zurückgebe
req = url + "Patient?_has:Observation:patient:code=8867-4&_count=5&_sort=-_lastUpdated"


response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "Bundle",
  "id": "1156f11e-6a3f-435b-9052-dbfa95254207",
  "meta": {
    "lastUpdated": "2022-12-13T14:27:08.219+00:00"
  },
  "type": "searchset",
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Patient?_count=5&_has%3AObservation%3Apatient%3Acode=8867-4&_sort=-_lastUpdated"
  }, {
    "relation": "next",
    "url": "https://hapi.fhir.org/baseR4?_getpages=1156f11e-6a3f-435b-9052-dbfa95254207&_getpagesoffset=5&_count=5&_pretty=true&_bundletype=searchset"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Patient/7198903",
    "resource": {
      "resourceType": "Patient",
      "id": "7198903",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2022-12-13T14:21:01.360+00:00",
        "source": "#79Y22jJNcs9cOtNc"
      },
      "text": {
        "status": "generated",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><div class=\"hapiHeaderText\">Fachhochschule <b>WIENERNEUSTADTFJEDR </b></div>

# Include

In [29]:
# _include lets you search for resource instances and include in the results other resources referenced by the target resource instances. 

req = url + 'Condition?code=http://snomed.info/sct|44054006' + '&' + '_include=Condition:patient'
response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "Bundle",
  "id": "0aa311fe-d31a-4f52-a08b-c4d99d9a1369",
  "meta": {
    "lastUpdated": "2022-12-13T14:27:10.141+00:00"
  },
  "type": "searchset",
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Condition?_include=Condition%3Apatient&code=http%3A%2F%2Fsnomed.info%2Fsct%7C44054006"
  }, {
    "relation": "next",
    "url": "https://hapi.fhir.org/baseR4?_getpages=0aa311fe-d31a-4f52-a08b-c4d99d9a1369&_getpagesoffset=20&_count=20&_pretty=true&_include=Condition%3Apatient&_bundletype=searchset"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Condition/601798",
    "resource": {
      "resourceType": "Condition",
      "id": "601798",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2020-02-07T11:03:45.714+00:00",
        "source": "#Io7DEVTOm0wT6Mkc"
      },
      "clinicalStatus": {
        "coding": [ {
          "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
          "code": 

# Reverse Include

In [30]:
#_revinclude allows you to search for resource instances and include in the results other resources that reference the target resource instances. 
req = url + 'Patient?name=WienerNeustadt&gender=female&birthdate=2022-12-05' + '&' + '_revinclude=Condition:patient'
response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "Bundle",
  "id": "c3a4a908-e344-46ba-beb0-ac731b6c898f",
  "meta": {
    "lastUpdated": "2022-12-13T14:27:10.541+00:00"
  },
  "type": "searchset",
  "total": 6,
  "link": [ {
    "relation": "self",
    "url": "https://hapi.fhir.org/baseR4/Patient?_revinclude=Condition%3Apatient&birthdate=2022-12-05&gender=female&name=WienerNeustadt"
  } ],
  "entry": [ {
    "fullUrl": "https://hapi.fhir.org/baseR4/Patient/7185197",
    "resource": {
      "resourceType": "Patient",
      "id": "7185197",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2022-12-05T15:33:01.605+00:00",
        "source": "#2eopjhS2NavITyoA"
      },
      "text": {
        "status": "generated",
        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><div class=\"hapiHeaderText\">Fachhochschule <b>WIENERNEUSTADTNJRLM </b></div><table class=\"hapiPropertyTable\"><tbody><tr><td>Date of birth</td><td><span>05 December 2022</span></td></tr></tbody></table></div>"
      },
      "na

# AddOn

In [31]:
# Daten exportieren, wird nicht von jedem FHIR-Server unterstützt
req = url + '$export' + '?' + '_type=Patient,Condition'
response = requests.request("GET", req, headers=headers)
print(response.text)

{
  "resourceType": "OperationOutcome",
  "text": {
    "status": "generated",
    "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><h1>Operation Outcome</h1><table border=\"0\"><tr><td style=\"font-weight: bold;\">ERROR</td><td>[]</td><td><pre>HAPI-0513: Must request async processing for $export</pre></td>\n\t\t\t</tr>\n\t\t</table>\n\t</div>"
  },
  "issue": [ {
    "severity": "error",
    "code": "processing",
    "diagnostics": "HAPI-0513: Must request async processing for $export"
  } ]
}


In [32]:
# Konfiguration des FHIR-Servers abrufen - wichtig um zu wissen, welche Ressourcen/ Funktionen untersützt werden.
# !! es muss nicht jeder FHIR-Server die gesamte Spezifikation unterstützen !!
req = url + 'metadata'
response = requests.request("GET", req, headers=headers)
print(response.text)

Output hidden; open in https://colab.research.google.com to view.

In [33]:
import pandas as pd

# Ergebnis (JSON) in Pandas Dataframe konvertieren
req = url + 'Patient' + '?' + '_has:Condition:patient:code=http://snomed.info/sct|44054006&_elements=id,gender,birthDate'
response = requests.request("GET", req, headers=headers)
#print(response.text)

data = json.loads(response.text)

df = pd.json_normalize(data, 'entry')
df.describe()

,fullUrl,resource.resourceType,resource.id,resource.meta.versionId,resource.meta.lastUpdated,resource.meta.source,resource.meta.tag,resource.gender,resource.birthDate,search.mode,resource.meta.profile
count,20,20,20,20,20,20,20,20,20,20,3
unique,20,1,20,3,20,20,2,2,8,1,1
top,https://hapi.fhir.org/baseR4/Patient/601740,Patient,601740,1,2020-02-07T11:03:45.714+00:00,#Io7DEVTOm0wT6Mkc,[{'system': 'http://terminology.hl7.org/CodeSy...,male,1990-11-24,match,[https://www.fhir.philips.com/4.0/StructureDef...
freq,1,20,1,17,1,1,19,18,7,20,3
